In [2]:
graph = {
    'A': {'B': 5, 'C': 2},
    'B': {'D': 1, 'E': 4},
    'C': {'F': 1, 'G': 7},
    'F': {'H': 4, 'I': 5},
    'D': {}, 'E': {}, 'G': {}, 'H': {}, 'I': {}
}


In [4]:
from collections import deque

def bfs(graph, start, goal):
    queue = deque([(start, [start], 0)])
    visited = set()
    
    while queue:
        node, path, cost = queue.popleft()
        if node == goal:
            return path, cost
        visited.add(node)
        for neighbor, weight in graph[node].items():
            if neighbor not in visited:
                queue.append((neighbor, path + [neighbor], cost + weight))
    return None, float('inf')


In [6]:
def dfs(graph, node, goal, path=[], cost=0, visited=None):
    if visited is None:
        visited = set()
    visited.add(node)
    path = path + [node]
    if node == goal:
        return path, cost
    for neighbor, weight in graph[node].items():
        if neighbor not in visited:
            result = dfs(graph, neighbor, goal, path, cost + weight, visited)
            if result[0]:
                return result
    return None, float('inf')


In [8]:
def dls(graph, node, goal, limit, path=[], cost=0):
    path = path + [node]
    if node == goal:
        return path, cost
    if limit <= 0:
        return None, float('inf')
    for neighbor, weight in graph[node].items():
        result = dls(graph, neighbor, goal, limit-1, path, cost + weight)
        if result[0]:
            return result
    return None, float('inf')


In [10]:
def ids(graph, start, goal, max_depth=10):
    for depth in range(max_depth):
        result = dls(graph, start, goal, depth)
        if result[0]:
            return result
    return None, float('inf')


In [12]:
import heapq

def ucs(graph, start, goal):
    pq = [(0, start, [start])]
    visited = set()

    while pq:
        cost, node, path = heapq.heappop(pq)
        if node == goal:
            return path, cost
        if node not in visited:
            visited.add(node)
            for neighbor, weight in graph[node].items():
                heapq.heappush(pq, (cost + weight, neighbor, path + [neighbor]))
    return None, float('inf')


In [14]:
def bidirectional(graph, start, goal):
    from collections import deque

    front = {start: (0, [start])}
    back = {goal: (0, [goal])}

    front_queue = deque([start])
    back_queue = deque([goal])

    while front_queue and back_queue:
        # Expand from start
        current = front_queue.popleft()
        current_cost, current_path = front[current]
        for neighbor, weight in graph[current].items():
            if neighbor not in front:
                front[neighbor] = (current_cost + weight, current_path + [neighbor])
                front_queue.append(neighbor)
            if neighbor in back:
                path1 = front[neighbor][1]
                path2 = back[neighbor][1]
                total_cost = front[neighbor][0] + back[neighbor][0]
                return path1 + path2[::-1][1:], total_cost

        # Expand from goal (reverse graph)
        for parent in graph:
            if current in graph[parent]:
                weight = graph[parent][current]
                if parent not in back:
                    back[parent] = (back[current][0] + weight, [parent] + back[current][1])
                    back_queue.append(parent)
                if parent in front:
                    path1 = front[parent][1]
                    path2 = back[parent][1]
                    total_cost = front[parent][0] + back[parent][0]
                    return path1 + path2[::-1][1:], total_cost

    return None, float('inf')


In [16]:
print("BFS       :", bfs(graph, 'A', 'D'))
print("DFS       :", dfs(graph, 'A', 'D'))
print("DLS (L=2) :", dls(graph, 'A', 'D', 2))
print("IDS       :", ids(graph, 'A', 'D'))
print("UCS       :", ucs(graph, 'A', 'D'))
print("Bi-Dir    :", bidirectional(graph, 'A', 'D'))


BFS       : (['A', 'B', 'D'], 6)
DFS       : (['A', 'B', 'D'], 6)
DLS (L=2) : (['A', 'B', 'D'], 6)
IDS       : (['A', 'B', 'D'], 6)
UCS       : (['A', 'B', 'D'], 6)
Bi-Dir    : (['A', 'B', 'D'], 6)
